## Importing  gcp libraries and setting credential

In [1]:
import argparse
import os
import time
from pprint import pprint
import googleapiclient.discovery
from six.moves import input

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/home/sathish/gcp_pem.json'
compute = googleapiclient.discovery.build('compute', 'v1')
project="ferrous-weaver-249914"
zone="us-east1-b"

## data nodes and master node Definition

In [2]:
data_node_instances={'datanode1':{'instance_name':'datanode1','hostname':'datanode1.tanu.com','script_template':'cloudera_datanode.sh'},
                   'datanode2':{'instance_name':'datanode2','hostname':'datanode2.tanu.com','script_template':'cloudera_datanode.sh'}}

master_node_instances={'masternode1':{'instance_name':'masternode','hostname':'master.tanu.com','script_template':'cloudera_manager.sh'}}


In [3]:
def generate_data_node_startup_scripts(master_node_name,master_node_ip):
    script = open('new-client-startup-script.sh', 'w') 
    print('''
    #!/bin/bash

    WORK_DIR="/var/tmp/startup_dir"
    
    mkdir $WORK_DIR
    cd $WORK_DIR
    hostname=$(hostname -f)
    echo "{master_node_ip} {master_node_name}" >>/etc/hosts
    yum -y install git-core net-tools krb5-workstation
    git clone https://github.com/skumarx87/openldap_MIT-Kerberos_installation.git
    cd openldap_MIT-Kerberos_installation
    sed -i "s/idm.tanu.com/{master_node_name}/g" kerberos_ldap_installation.sh
    ./kerberos_ldap_installation.sh client_setup
    '''.format(master_node_ip=master_node_ip,master_node_name=master_node_name),file=script)
    script.close()

def generate_master_node_startup_scripts(master_node_name,script_template):
    script = open('new-server-startup-script.sh', 'w') 
    print('''
    #!/bin/bash
    WORK_DIR="/var/tmp/startup_dir"
    mkdir $WORK_DIR
    cd $WORK_DIR
    echo "this is startup script"
    yum -y install git-core net-tools krb5-workstation
    git clone https://github.com/skumarx87/openldap_MIT-Kerberos_installation.git
    cd openldap_MIT-Kerberos_installation
    sed -i "s/idm.tanu.com/{master_node_name}/g" kerberos_ldap_installation.sh
    ./kerberos_ldap_installation.sh server_setup
    ./kerberos_ldap_installation.sh setup_webserver
    ./kerberos_ldap_installation.sh client_setup
    ./kerberos_ldap_installation.sh create_hadoop_users
    chmod 755 {script_template}
    sh {script_template} |tee -a cm_install.log

    '''.format(master_node_name=master_node_name,script_template=script_template),file=script)
    script.close()
    
def list_instances():
        result = compute.instances().list(project=project, zone=zone).execute()
        return result['items'] if 'items' in result else None
        # [END list_instances]

def list_images():
        request = compute.images().list(project=project).execute()
        print(request)
        #while request is not None:
        #       response = request.execute()
        #       print(response)
                #for image in response['items']:
                #       pprint(image)
        #       request = compute.images().list_next(previous_request=request, previous_response=response)
def list_disks():
        request = compute.disks().list(project=project, zone=zone)
        while request is not None:
            response = request.execute()
            for disk in response['items']:
                pprint(disk)
            request = service.disks().list_next(previous_request=request, previous_response=response)
        
        
def get_instance_ip(instance):
        request = compute.instances().get(project=project, zone=zone, instance=instance)
        response = request.execute()
        pprint(response)
        host_ip=response['networkInterfaces'][0]['networkIP']
        return host_ip
        
def create_instance(name,hostname,startup_script):
        image_response = compute.images().getFromFamily(project='centos-cloud', family='centos-7').execute()
        source_disk_image = image_response['selfLink']
        machine_type = "zones/%s/machineTypes/n1-standard-1" % zone
        startup_script = open(
                os.path.join(
                        os.path.abspath(''), startup_script), 'r').read()
        config = {
            'name': name,
            'machineType': machine_type,
            'hostname':hostname,
            'disks': [
                {
                    'boot': True,
                    'autoDelete': True,
                    'initializeParams': {
                        'sourceImage': source_disk_image,
                        'diskSizeGb': 10,
                    }
                }
            ],
            # Specify a network interface with NAT to access the public
            # internet.
            'networkInterfaces': [{
                'network': 'global/networks/default',
                'accessConfigs': [
                    {'type': 'ONE_TO_ONE_NAT', 'name': 'External NAT'}
                ]
            }],
            'metadata': {
            'items': [{
                # Startup script is automatically executed by the
                # instance upon startup.
                'key': 'startup-script',
                'value': startup_script
            }]
            }
        }
        return compute.instances().insert(project=project,zone=zone,body=config).execute()
            
def wait_for_operation(operation):
    print('Waiting for operation to finish...')
    while True:
        result = compute.zoneOperations().get(
            project=project,
            zone=zone,
            operation=operation).execute()

        if result['status'] == 'DONE':
            print("done.")
            if 'error' in result:
                raise Exception(result['error'])
            return result

        time.sleep(1)


## Inserting Firewall rule to allow cloudera Manager Port 7180 from internet

In [4]:
def insert_filrewall_rule(name,ports):
    firewall_body={
        'name':name,
        'allowed':[
            {
                "IPProtocol":'tcp',
                "ports":ports
            }
        ],
        "sourceRanges": ['0.0.0.0/0']
    }
    request = compute.firewalls().insert(project=project, body=firewall_body)
    response = request.execute()
    pprint(response)

In [5]:
def Create_hadoop_instaces(wait=True):
        #compute = googleapiclient.discovery.build('compute', 'v1')
        print("Creating Master node")
                     
        for masternode in master_node_instances:
            inst_name=master_node_instances[masternode]['instance_name']
            host_name=master_node_instances[masternode]['hostname']
            script_template=master_node_instances[masternode]['script_template']
            generate_master_node_startup_scripts(host_name,script_template)
            operation=create_instance(inst_name,host_name,'new-server-startup-script.sh')
            print(operation)
            wait_for_operation(operation['name'])

        master_node_inst_name=master_node_instances['masternode1']['instance_name']
        master_node_host_name=master_node_instances['masternode1']['hostname']
        master_node_ip=get_instance_ip(master_node_inst_name)
        time.sleep(80) ## wait for master node startup script completion
        for datanode in data_node_instances:
            print("Creating Data Node : {}".format(datanode))
            inst_name=data_node_instances[datanode]['instance_name']
            host_name=data_node_instances[datanode]['hostname']
            generate_data_node_startup_scripts(master_node_host_name,master_node_ip)
            create_instance(inst_name,host_name,'new-client-startup-script.sh')
            


In [6]:
if __name__ == '__main__':
        Create_hadoop_instaces()
        insert_filrewall_rule("cloudera-manager-url",["7180"])

Creating Master node
{'id': '4325156374218806305', 'name': 'operation-1576643277758-599f2de822098-16a5e206-a38c6bd0', 'zone': 'https://www.googleapis.com/compute/v1/projects/ferrous-weaver-249914/zones/us-east1-b', 'operationType': 'insert', 'targetLink': 'https://www.googleapis.com/compute/v1/projects/ferrous-weaver-249914/zones/us-east1-b/instances/masternode', 'targetId': '1851842368111907873', 'status': 'RUNNING', 'user': 'libcloud@ferrous-weaver-249914.iam.gserviceaccount.com', 'progress': 0, 'insertTime': '2019-12-17T20:27:58.908-08:00', 'startTime': '2019-12-17T20:27:58.910-08:00', 'selfLink': 'https://www.googleapis.com/compute/v1/projects/ferrous-weaver-249914/zones/us-east1-b/operations/operation-1576643277758-599f2de822098-16a5e206-a38c6bd0', 'kind': 'compute#operation'}
Waiting for operation to finish...
done.
{'cpuPlatform': 'Intel Haswell',
 'creationTimestamp': '2019-12-17T20:27:58.813-08:00',
 'deletionProtection': False,
 'disks': [{'autoDelete': True,
            'boo